In [1]:
import sys

sys.path.insert(0,'../sib/')
import csv
import sib
import os
import numpy as np
import pandas as pd
import sklearn.metrics as mm
# sir_inference imports
from sir_model import FastProximityModel, patient_zeros_states
from ranking import csr_to_list
import os.path
from os import path
N=500000
#N = 50000
## new try with 100 spreaders
N_patient_zero = 200;
lamb_load = 0.05;
lamb_true = 0.05;
mu_true = 0.02;


################# parameters used in the inference proc. ###################
lamb_inf = 0.02;
mu_inf = 0.03; 
############################################################################

scale=1.0; # Easy Case
T=100;
seed=5;
location="networks"
# SIR parameters
t1 = 10;
#seed=int();


Bad key "text.kerning_factor" on line 4 in
/opt/anaconda3/envs/bigdatalab_cpu_202101/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.2/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
## import sys
from time import time
import numpy as np
import pandas as pd
import os.path
from os import path
from scenario import Scenario
from sir_model import EpidemicModel, patient_zeros_states, symptomatic_individuals
from ranking import RANKINGS
log_fname="interactions_proximity_N%dK_s%.1f_T%d_lamb%.2f_s%d.csv"%(N/1000,scale,T,lamb_load,seed)
csv_file=location+"/"+log_fname
print("seed = %d"%seed, flush=True)
np.random.seed(seed);
if path.exists(csv_file): print("Loading model from %s"%csv_file)
else:
    print("Could not find the file! Was looking for \n "+csv_file+"\n Bye-Bye")
    sys.exit()
print("Load Proximity model", flush=True)
tic = time()
model = EpidemicModel(initial_states=np.zeros(N), x_pos=np.zeros(N), y_pos=np.zeros(N))
model.load_transmissions(csv_file, new_lambda = lamb_true)
model.recover_probas = mu_true*np.ones(N)
print(f"Loading took {time()-tic:.1f}s",flush=True)
model.initial_states = patient_zeros_states(N, N_patient_zero)
model.time_evolution(model.recover_probas, model.transmissions, print_every=50)
t_max = len(model.transmissions)
print("Save plain dynamics", flush=True)
db = pd.DataFrame()
db["S"] = np.sum(model.states==0,axis=1)
db["I"] = np.sum(model.states==1,axis=1)
db["R"] = np.sum(model.states==2,axis=1)
db.to_csv("csv/Proximity_N%dK_T%d_s1_pz%d_mu%.2f_l%.2f_seed%d.csv"%(N/1000,T,N_patient_zero,mu_true,lamb_true,seed),
          index=False, sep="\t")
del db


model.initial_states = model.states[t1]
model.states = model.states[t1:]
model.transmissions = model.transmissions[t1:]

seed = 5
Loading model from networks/interactions_proximity_N500K_s1.0_T100_lamb0.05_s5.csv
Load Proximity model
Loading transmissions from networks/interactions_proximity_N500K_s1.0_T100_lamb0.05_s5.csv
Loading took 302.1s
Running SIR simulation
t = 0 / 100
t = 50 / 100
Save plain dynamics


In [3]:
trac_lambd = lamb_inf
trac_tau = 5
# MF parameters 
MF_tau = 5;
MF_delta = 15;
# observation parameters
n_ranking = 1500
#n_ranking = 500
p_untracked=0
#seed = int(sys.argv[2]);
#seeds for running [32,123,456]
#seed=int(sys.argv[1]);
################################################

intervention_options=dict(quarantine_time=T-t1)
observation_options=dict(n_random=0,n_infected=0,n_ranking=n_ranking, p_symptomatic=0.5, tau=5, p_untracked=p_untracked)

In [ ]:
import imp
import scenario
imp.reload(scenario)
from scenario import Scenario
##### TRACING UP TO SECOND NN #######
scenario_trac_2nd = Scenario( model, seed=seed+1, 
    ranking_options=dict(ranking=RANKINGS["tracing2nd"], tau=trac_tau, lamb=trac_lambd),
    observation_options=observation_options,
    intervention_options=intervention_options
)
scenario_trac_2nd.run(t_max-t1, print_every = 1)
print("Save tracing sec NN strategy", flush=True)
scenario_trac_2nd.counts.to_csv("csv/Proximity_N%dK_T%d_s1_ti%d_pz%d_mu%.2f_l%.2f_seed%d_obs%d_trac2nd_t%d_mism.csv"%(N/1000,T,t1,N_patient_zero,mu_inf,lamb_inf,seed,n_ranking,trac_tau),
                  index=False, sep="\t")
#scenario_trac_2nd.save("csv/Proximity_N%dK_T%d_s1_ti%d_pz%d_mu%.2f_l%.2f_seed%d_obs%d_trac2nd_t%d_all"%(N/1000,T,t1,N_patient_zero,mu,lamb,seed,n_ranking,trac_tau))

del scenario_trac_2nd
print("End seed", flush=True)

Using ranking_tracing_secnn to rank
Using seed=6
Selecting 0/500000 untracked individuals
t = 0 / 90
state: S:497821, I:2036, R:143 I+R:2179
obser: S:1494, I:5, R:1, I+R:6
free birds: 2031
obs_symp:0(0), obs_rank:5(5)
t = 1 / 90
state: S:497344, I:2482, R:174 I+R:2656
obser: S:2991, I:8, R:1, I+R:9
free birds: 2474
obs_symp:0(0), obs_rank:3(8)
t = 2 / 90
state: S:496814, I:2951, R:235 I+R:3186
obser: S:4483, I:16, R:1, I+R:17
free birds: 2935
obs_symp:0(0), obs_rank:8(16)
t = 3 / 90
state: S:496169, I:3528, R:303 I+R:3831
obser: S:5967, I:32, R:1, I+R:33
free birds: 3496
obs_symp:0(0), obs_rank:16(32)
t = 4 / 90
state: S:495465, I:4163, R:372 I+R:4535
obser: S:7455, I:43, R:2, I+R:45
free birds: 4120
obs_symp:0(0), obs_rank:11(43)
Appeding contacts, t=5
Appeding contacts, t=4
Appeding contacts, t=3
Appeding contacts, t=2
Appeding contacts, t=1
Appeding contacts, t=0
first NN c: 984. second NN c: 16793
t = 5 / 90
state: S:494700, I:4846, R:454 I+R:5300
obser: S:8456, I:500, R:44, I+R:54

In [ ]:
##### MF SCENARIO #######
import imp
import scenario
imp.reload(scenario)
from scenario import Scenario
import ranking
imp.reload(ranking)
from ranking import RANKINGS
scenario_MF = Scenario(
    model, seed=seed+1, 
    ranking_options=dict(ranking=RANKINGS["backtrack"], 
                         algo="MF", init="all_S", tau=MF_tau, delta=MF_delta, mu=mu_inf, lamb=lamb_inf), 
    observation_options=observation_options,
    intervention_options=intervention_options,
)
scenario_MF.run(t_max-t1, print_every = 1)
print("Save MF strategy", flush=True)
scenario_MF.counts.to_csv("csv/Proximity_N%dK_T%d_s1_ti%d_pz%d_mu%.2f_l%.2f_seed%d_obs%d_MF_t%d_d%d_mism.csv"%(N/1000,T,t1,N_patient_zero,mu_inf,lamb_inf,seed,n_ranking,MF_tau,MF_delta),
                  index=False, sep="\t")
#scenario_MF.save("csv/Proximity_N%dK_T%d_s1_ti%d_pz%d_mu%.2f_l%.2f_seed%d_obs%d_MF_t%d_d%d_all"%(N/1000,T,t1,N_patient_zero,mu_inf,lamb_inf,seed,n_ranking,MF_tau,MF_delta))

del scenario_MF
# 1h01min per round
print("End seed", flush=True)

In [ ]:
import imp
import scenario
imp.reload(scenario)
from scenario import Scenario
import sib
import bp_ranking
imp.reload(bp_ranking)
from bp_ranking import bp_ranker_class

mu_r = np.log(1 + mu_inf)
win = 21
#bp_tau = 10
name_csv = "csv/Proximity_N%dK_T%d_s1_ti%d_pz%d_mu%.2f_l%.2f_seed%d_obs%d_bp_win%d_mism.csv"%(N/1000, T,t1,N_patient_zero,mu_inf,lamb_inf,seed,n_ranking,win)
prob_seed=1/N
prob_sus = 0.55
pseed = prob_seed / (2 - prob_seed)
psus = prob_sus * (1 - pseed)
pautoinf = 1e-10

bp_ranker = bp_ranker_class(params = sib.Params(
                                 prob_r = sib.Exponential(mu=mu_r),
                                 prob_i = sib.Uniform(p=lamb_inf),
                                 pseed = pseed,
                                 psus = psus,
                                 pautoinf = pautoinf,
                                 fn_rate = 0.001,
                                 fp_rate = 0.001),
                                 
                 maxit0 = 30,
                 maxit1 = 30,
                 tol = 1e-3,
                 memory_decay = 0.1,
                 window_length = win
                            #tau=bp_tau
                           )
bp_ranker.init(N, T)
bp_ranker.__name__ = "bp"
scenario_bp = Scenario( model, seed=seed+1, 
    ranking_options=dict(ranking=bp_ranker.step_scenario, lamb = 1.0),
    observation_options=observation_options,
    intervention_options=intervention_options,
                         save_csv = name_csv
)
scenario_bp.run(t_max-t1, print_every = 1)
print("Save bp strategy", flush=True)
scenario_bp.counts.to_csv(name_csv,
          index=False, sep="\t")
